<!-- # Tutorial Overview
This tutorial is divided into five parts; they are:

1. Problem of Multioutput Regression
    - Check Scikit-Learn Version
    - Multioutput Regression Test Problem
2. Inherently Multioutput Regression Algorithms
    - Linear Regression for Multioutput Regression
    - k-Nearest Neighbors for Multioutput Regression
    - Evaluate Multioutput Regression With Cross-Validation
3. Wrapper Multioutput Regression Algorithms
4. Direct Multioutput Regression
5. Chained Multioutput Regression -->

Source: [machinelearningmastery multi-output-regression][1]

[1]: https://machinelearningmastery.com/multi-output-regression-models-with-python/

## Problem of Multioutput Regression
Regression refers to a predictive modeling problem that involves predicting a numerical value.

For example, predicting a size, weight, amount, number of sales, and number of clicks are regression problems. Typically, a single numeric value is predicted given input variables.

Some regression problems require the prediction of two or more numeric values. For example, predicting an x and y coordinate. These problems are referred to as multiple-output regression, or multioutput regression.

- **Regression:** Predict a single numeric output given an input.
- **Multioutput Regression:** Predict two or more numeric outputs given an input.

In multioutput regression, typically the outputs are dependent upon the input and upon each other. This means that often the outputs are not independent of each other and may require a model that predicts both outputs together or each output contingent upon the other outputs.

Multi-step time series forecasting may be considered a type of multiple-output regression where a sequence of future values are predicted and each predicted value is dependent upon the prior values in the sequence.

In [1]:
import sklearn
print(sklearn.__version__)

1.3.0


We will use the make_regression() function to create a test dataset for multiple-output regression. We will generate 1,000 examples with 10 input features, five of which will be redundant and five that will be informative. The problem will require the prediction of two numeric values.

- **Problem Input**: 10 numeric variables.
- **Problem Output**: 2 numeric variables.

In [2]:
from sklearn.datasets import make_regression
X,y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
print(X.shape, y.shape)

(1000, 10) (1000, 2)


Some regression machine learning algorithms support multiple outputs directly. This includes most of the popular machine learning algorithms implemented in the scikit-learn library, such as:

- LinearRegression (and related)
- KNeighborsRegressor
- DecisionTreeRegressor
- RandomForestRegressor (and related)

### Linear Regression for Multioutput Regression

In [5]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(X, y)
X_test = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
y_lr_pred = model_lr.predict([X_test])
print(y_lr_pred)


[[50.06781717 64.564973  ]]


### k-Nearest Neighbors for Multioutput Regression

In [7]:
from sklearn.neighbors import KNeighborsRegressor
model_knr = KNeighborsRegressor()
model_knr.fit(X, y)
y_knr_pred = model_knr.predict([X_test])
print(y_knr_pred)

[[-11.73511093  52.78406297]]


### Decision Tree for Multioutput Regression

In [8]:
from sklearn.tree import DecisionTreeRegressor
model_dtr = DecisionTreeRegressor()
model_dtr.fit(X,y)
y_dtr_pred = model_dtr.predict([X_test])
print(y_dtr_pred)

[[49.93137149 64.08484989]]


## Evaluate Multioutput Regression With Cross-Validation

In [12]:
from numpy import absolute, mean, std
from sklearn.model_selection import cross_val_score, RepeatedKFold
# create datasets

model = DecisionTreeRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 52.285 (3.407)


## Wrapper Multioutput Regression Algorithms
Not all regression algorithms support multioutput regression.

One example is the support vector machine, although for regression, it is referred to as support vector regression, or SVR.

This algorithm does not support multiple outputs for a regression problem and will raise an error. We can demonstrate this with an example, listed below.

In [13]:
# failure of support vector regression for multioutput regression (causes an error)
from sklearn.datasets import make_regression
from sklearn.svm import LinearSVR
# create datasets
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1)
# define model
model = LinearSVR()
# fit model
# (THIS WILL CAUSE AN ERROR!)
model.fit(X, y)

ValueError: y should be a 1d array, got an array of shape (1000, 2) instead.

Running the example reports (`ValueError: bad input shape (1000, 2)`) an error message indicating that the model does not support multioutput regression.

(all text are copied from [machinelearningmastery multi-output-regression][1])

[1]: https://machinelearningmastery.com/multi-output-regression-models-with-python/

A workaround for using regression models designed for predicting one value for multioutput regression is to divide the multioutput regression problem into multiple sub-problems.

The most obvious way to do this is to split a multioutput regression problem into multiple single-output regression problems.

For example, if a multioutput regression problem required the prediction of three values y1, y2 and y3 given an input X, then this could be partitioned into three single-output regression problems:

- **Problem 1**: Given X, predict y1.
- **Problem 2**: Given X, predict y2.
- **Problem 3**: Given X, predict y3.

There are two main approaches to implementing this technique.

### The first approach is good for neighbours 
The first approach involves developing a separate regression model for each output value to be predicted. We can think of this as a direct approach, as each target value is modeled directly.


### The second approach is good for 
The second approach is an extension of the first method except the models are organized into a chain. The prediction from the first model is taken as part of the input to the second model, and the process of output-to-input dependency repeats along the chain of models.

- **Direct Multioutput**: Develop an independent model for each numerical value to be predicted.
- **Chained Multioutput**: Develop a sequence of dependent models to match the number of numerical values to be predicted.
Let’s take a closer look at each of these techniques in turn.

## Direct Multioutput Regression

The direct approach to multioutput regression involves dividing the regression problem into a separate problem for each target variable to be predicted.

This assumes that the outputs are independent of each other, which might not be a correct assumption. Nevertheless, this approach can provide surprisingly effective predictions on a range of problems and may be worth trying, at least as a performance baseline.

For example, the outputs for your problem may, in fact, be mostly independent, if not completely independent, and this strategy can help you find out.

This approach is supported by the [MultiOutputRegressor][2] class that takes a regression model as an argument. It will then create one instance of the provided model for each output in the problem.

[2]: https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html

```
# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
```


In [14]:
# example of evaluating direct multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.419 (0.024)


We can also use the direct multioutput regression wrapper as a final model and make predictions on new data.

First, the model is fit on all available data, then the predict() function can be called to make predictions on new data

In [15]:
# example of making a prediction with the direct multioutput regression model
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
# fit the model on the whole dataset
wrapper.fit(X, y)
# make a single prediction
X_test = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
y_wr_pred = wrapper.predict([X_test])
# summarize the prediction
print('Predicted: %s' % y_wr_pred)

g:\github_projects\study-time-series-analysis\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
g:\github_projects\study-time-series-analysis\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Predicted: [[50.04979758 64.50953349]]


## Chained Multioutput Regression
Another approach to using single-output regression models for multioutput regression is to create a linear sequence of models.

The first model in the sequence uses the input and predicts one output; the second model uses the input and the output from the first model to make a prediction; the third model uses the input and output from the first two models to make a prediction, and so on.

For example, if a multioutput regression problem required the prediction of three values y1, y2 and y3 given an input X, then this could be partitioned into three dependent single-output regression problems as follows:

- **Problem 1**: Given X, predict y1.
- **Problem 2**: Given X and yhat1, predict y2.
- **Problem 3**: Given X, yhat1, and yhat2, predict y3.

This can be achieved using the [RegressorChain][3] class in the scikit-learn library.

[3]: https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.RegressorChain.html

The order of the models may be based on the order of the outputs in the dataset (the default) or specified via the “order” argument. For example, order=[0,1] would first predict the oth output, then the 1st output, whereas an order=[1,0] would first predict the last output variable and then the first output variable in our test problem.

The example below demonstrates how we can first create a single-output regression model then use the RegressorChain class to wrap the regression model and add support for multioutput regression.

```
# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
wrapper = RegressorChain(model, order=[0,1])
```

In [16]:
# example of evaluating chained multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.611 (0.268)


In [17]:
# example of making a prediction with the chained multioutput regression model
from sklearn.datasets import make_regression
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# fit the model on the whole dataset
wrapper.fit(X, y)
# make a single prediction
X_test = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
y_wr2_pred = wrapper.predict([X_test])
# summarize the prediction
print('Predicted: %s' % y_wr2_pred)

Predicted: [[50.017815   65.04742335]]


g:\github_projects\study-time-series-analysis\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
g:\github_projects\study-time-series-analysis\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
g:\github_projects\study-time-series-analysis\venv\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
